In [2]:
import os
import sys
from dotenv import load_dotenv

# für später wenn ich rag evaluiere 
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path since we work with notebooks
# from helper_functions import *
#from evaluation.evalute_rag import *
from embeddings import *
from llms import *

# Initialisiere die Modelle
# mistral_model, llama_model = initialize_models()

# Nutze die Modelle weiter
# print(mistral_model)
# print(llama_model)

# Load environment variables from a .env file
load_dotenv()
#from rank_bm25 import BM25Okapi

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.documents import Document
from sentence_transformers import SentenceTransformer

from langchain.embeddings.base import Embeddings
from typing import List

import numpy as np

import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings


/Volumes/data1/Programming/predictiv_chatbot/predictiv_chatbot/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [21]:
def replace_t_with_space(list_of_documents):
    """
    Replaces all tab characters ('\t') with spaces in the page content of each document.

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces.
    """

    for doc in list_of_documents:
        doc.page_content = doc.page_content.replace('\t', ' ')  # Replace tabs with spaces
    return list_of_documents

In [3]:
def delete_chroma_collection(collection_name='document_embeddings_v10', persist_directory='chroma_db_v4'):
    # Öffne die Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Überprüfe, ob die Collection existiert, und lösche sie, wenn ja
    existing_collections = [col.name for col in client.list_collections()]
    if collection_name in existing_collections:
        client.delete_collection(name=collection_name)
        print(f"Collection '{collection_name}' erfolgreich gelöscht.")
    else:
        print(f"Collection '{collection_name}' nicht gefunden.")
        
delete_chroma_collection(collection_name='document_embeddings_v10')

Collection 'document_embeddings_v10' erfolgreich gelöscht.


In [26]:
def encode_md(path, chunk_size=500, chunk_overlap=100):
    """
    Encodes a Markdown file into a vector store using OpenAI embeddings.

    Args:
        path: The path to the Markdown file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        
    """

    # Load Markdown documents
    loader = UnstructuredMarkdownLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)
    # print(cleaned_documents)
    # Create embeddings and vector store
    # Extrahiere den Text aus den Document-Objekten
    # texts = [doc.page_content for doc in cleaned_texts]

    # Erstelle die Embeddings
    # embeddings = model.encode(texts, convert_to_numpy=True)
    # print(cleaned_texts)
    
    # Gliederung der Dokumente
    # for document in cleaned_texts:
    #     print(f"Quelle: {document.metadata['source']}")
    #     print("\nInhalt:")
    #     print(document.page_content)
    #     print("\n" + "-"*50 + "\n")
    
    return cleaned_texts


md_folder_path = os.path.join('data', 'md_cleaned', 'was_kommt_wo_hinein_de.md')


chunks_vector_store = encode_md(md_folder_path, chunk_size=500, chunk_overlap=100)

In [27]:
def encode_md_from_folder(folder_path, chunk_size=500, chunk_overlap=100):
    """
    Encodes all Markdown files in a folder into a vector store using OpenAI embeddings.

    Args:
        folder_path: The path to the folder containing Markdown files.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A list of cleaned text chunks from all Markdown files in the folder.
    """

    all_cleaned_texts = []

    # Iterate through all files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".md"):
            file_path = os.path.join(folder_path, filename)

            # Load Markdown documents
            loader = UnstructuredMarkdownLoader(file_path)
            documents = loader.load()

            # Split documents into chunks
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
            )
            texts = text_splitter.split_documents(documents)
            cleaned_texts = replace_t_with_space(texts)

            # Append cleaned texts to the list
            all_cleaned_texts.extend(cleaned_texts)

    return all_cleaned_texts

# Beispielverwendung
md_folder_path = os.path.join('data', 'md_cleaned')

chunks_vector_store = encode_md_from_folder(md_folder_path, chunk_size=500, chunk_overlap=100)

In [28]:
from IPython.display import display, HTML

def print_corrections(original, corrected):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Quelle:</h3>
        <p style="white-space: pre-wrap;">{original}</p>
        <h3>Inhalt Chunk:</h3>
        <p style="white-space: pre-wrap; color: green;">{corrected}</p>
    </div>
    """))
    
# Aufruf der Funktion für jedes Dokument

    
for document in chunks_vector_store:
    print_corrections({document.metadata['source']}, document.page_content)
    

In [5]:
def add_embeddings(list_of_documents):
    """

    Args:
        list_of_documents: A list of document objects, each with a 'page_content' attribute.

    Returns:
        The modified list of documents with tab characters replaced by spaces 
        and embeddings added.
    """
    docs_with_embeddings_2 = []
    
    for doc in list_of_documents:
        
        # Generate embeddings for the page content
        embeddings = create_embeddings(doc.page_content)

        # Store the document and its embeddings in a dictionary
        doc_dict = {'metadata': doc.metadata ,'page_content': doc.page_content, 'embeddings': embeddings}
        docs_with_embeddings_2.append(doc_dict)
    
    return docs_with_embeddings_2
    

docs_with_embeddings = add_embeddings(chunks_vector_store)

# Gliederung der Dokumente
# for document in docs_with_embeddings:
#     print(f"Quelle: {document['metadata']['source']}")
#     print("\nInhalt:")
#     print(document['page_content'])
#     print("\nEmbeddings:")
#     print(document['embeddings'])
#     print("\n" + "-"*50 + "\n")

Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt
Embedding erstellt


In [ ]:
from IPython.display import display, HTML

def print_corrections(original, corrected, emb):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Quelle:</h3>
        <p style="white-space: pre-wrap;">{original}</p>
        <h3>Inhalt Chunk:</h3>
        <p style="white-space: pre-wrap; color: green;">{corrected}</p>
        <h3>Embeddings:</h3>
        <p style="white-space: pre-wrap; color: yellow;">{emb}</p>
    </div>
    """))
    
# Aufruf der Funktion für jedes Dokument

    
for document in docs_with_embeddings:
    print_corrections({document['metadata']['source']}, document['page_content'], document['embeddings'])


In [ ]:
for document in docs_with_embeddings:
    print({document['metadata']['source']})

In [6]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings
#from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer


def create_chroma_v2(documents, collection_name, persist_directory):
    # Erstelle oder öffne eine Chroma-Datenbank mit Persistierung
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    # Erstelle eine neue Collection in der Datenbank oder lade eine bestehende
    collection = client.get_or_create_collection(name=collection_name)
    
    print(f"Chroma-Datenbank '{collection_name}' erfolgreich erstellt.")

    for i, chunk in enumerate(documents):
        # IDs werden dynamisch erzeugt, z.B. basierend auf dem Index
        doc_id = f"id_{i+1}"

        # Konvertiere die Embeddings in eine Liste von Listen
            # Extrahiere die Metadaten, den Seiteninhalt und die Embeddings aus dem Dokument
        source = chunk['metadata']['source']
        page_content = chunk['page_content']
        embeddings = chunk['embeddings']
        
        # Sicherstellen, dass Embeddings in der richtigen Form vorliegen (z.B. Liste von Listen)
        if isinstance(embeddings, np.ndarray):  # Falls die Embeddings als NumPy-Array vorliegen
            embeddings = embeddings.tolist()
            
        # Sicherstellen, dass page_content ein String ist
        if isinstance(page_content, Document):
            page_content = str(page_content)

        # Optional: Keywords aus einer anderen Quelle beziehen, falls nötig
        # keywords = keywords_per_chunk[i]  # Wenn du weiterhin auf die Keywords zugreifen möchtest

        # Beispiel-Keyword-String (falls du weiterhin einen benötigen solltest)
        # keywords_str = ", ".join(keywords)  # Dies kannst du anpassen, je nach Bedarf

        # Hinzufügen des Chunks mit den entsprechenden Metadaten und Embeddings
        collection.add(
            documents=[page_content],  # Der Seiteninhalt des Dokuments
            metadatas=[{"source": source}],  # Die Quelle als Metadaten
            ids=[doc_id],  # Eine eindeutige ID für jeden Eintrag
            embeddings=embeddings  # Die Embeddings als Liste von Listen
        )
        
        print(doc_id)

    print("Alle Chunks wurden erfolgreich mit ihren Embeddings und Metadaten zu Chroma hinzugefügt.")
    
    
collection_name='document_embeddings_v10'
persist_directory='chroma_db_v4'


create_chroma_v2(docs_with_embeddings, collection_name, persist_directory)

Chroma-Datenbank 'document_embeddings_v10' erfolgreich erstellt.
id_1
id_2
id_3
id_4
id_5
id_6
id_7
id_8
id_9
id_10
id_11
id_12
id_13
id_14
id_15
id_16
Alle Chunks wurden erfolgreich mit ihren Embeddings und Metadaten zu Chroma hinzugefügt.


In [17]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def get_first_document_from_chroma(collection_name, persist_directory):
    # Erstelle einen PersistentClient, um die Chroma-Datenbank zu öffnen
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Lade die entsprechende Collection
    collection = client.get_collection(name=collection_name)
    
    # Definiere die ID des ersten Dokuments (basierend auf deiner vorherigen Logik)
    first_doc_id = "id_4"
    
    # Suche nach dem ersten Dokument in der Collection
    results = collection.get(ids=[first_doc_id])
    
    # Gib das Dokument und seine Metadaten aus
    if results:
        document = results['documents'][0]
        metadata = results['metadatas'][0]
        ids = results['ids'][0]
        print("ID:", ids)
        print("Erstes Dokument:")
        print("Inhalt:", document)
        print("Metadaten:", metadata)
    else:
        print("Kein Dokument mit der ID 'id_1' gefunden.")

# Beispielaufruf der Funktion
collection_name='document_embeddings_v10'
persist_directory='chroma_db_v4'
get_first_document_from_chroma(collection_name, persist_directory)

ID: id_4
Erstes Dokument:
Inhalt: Darüber hinaus gibt es folgende Entsorgungsangebote:

Altglas:
Altglascontainer für Flaschen und Deckelgläser finden Sie an öffentlichen Plätzen.

Altkleider:
Container für Altkleider und Schuhe finden Sie in jedem Stadtteil auf öffentlichen Plätzen sowie auf den FES-Wertstoffhöfen.
Metadaten: {'source': 'data/md_cleaned/was_kommt_wo_hinein_de.md'}


In [3]:
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

def get_similar_documents_from_chroma(collection_name, persist_directory, query, top_k=3):
    # Erstelle einen PersistentClient, um die Chroma-Datenbank zu öffnen
    client = chromadb.PersistentClient(
        path=persist_directory,
        settings=Settings(),
        tenant=DEFAULT_TENANT,
        database=DEFAULT_DATABASE,
    )
    
    # Lade die entsprechende Collection
    collection = client.get_collection(name=collection_name)
    
    # Erstelle die Embeddings für die Abfrage
    query_embeddings = create_query_embeddings(query).tolist()
    
    # Führe die Abfrage in der Collection durch und suche die ähnlichsten Dokumente
    results = collection.query(query_embeddings=query_embeddings, n_results=top_k)
    
    # Überprüfe den Aufbau des Ergebnisses und gib die Dokumente aus
    if 'documents' in results and 'metadatas' in results and 'distances' in results:
        for i, (document, metadata, distance) in enumerate(zip(results['documents'][0], results['metadatas'][0], results['distances'][0])):
            similarity_score = 1 / (1 + distance)  # Berechnung des Ähnlichkeitsscores
            print("Ähnlichkeitsscore:", similarity_score)
            print(f"Dokument {i+1}:")
            print("Inhalt:", document)
            print("Metadaten:", metadata)

            print()
    else:
        print("Keine ähnlichen Dokumente gefunden.")
        
    return results

# Beispielaufruf der Funktion
# query = "Wie entsorge ich mein Altglas?"
# query = "Wie lautet die Telefonummer der FES?"
query = "Wo entsorge ich meine Plastikverpackungen?"

collection_name='document_embeddings_v10'
persist_directory='chroma_db_v4'
rag_results = get_similar_documents_from_chroma(collection_name, persist_directory, query)

Embedding erstellt
Ähnlichkeitsscore: 0.7550992051784692
Dokument 1:
Inhalt: Geschäumte Kunststoffe, z. B. Obst- und Gemüseverpackungen

Das kommt nicht hinein:

Verpackungen aus Papier und Pappe (Altpapiertonne)

Verpackungen aus Glas (Altglascontainer)

Stark verschmutzte Fast-Food-Verpackungen wie Pizzakartons (Restmüll)

Hinweis: Ist zu viel Restmüll in der gelben Tonne, können die Wertstoffe nicht mehr aussortiert und weiterverarbeitet werden.

Standort: Am Wohnhaus
Leerung: 14-täglich

Die grüne Altpapiertonne

Das kommt hinein:
Metadaten: {'source': 'data/md_cleaned/was_kommt_wo_hinein_de.md'}

Ähnlichkeitsscore: 0.7531164559257947
Dokument 2:
Inhalt: Darüber hinaus gibt es folgende Entsorgungsangebote:

Altglas:
Altglascontainer für Flaschen und Deckelgläser finden Sie an öffentlichen Plätzen.

Altkleider:
Container für Altkleider und Schuhe finden Sie in jedem Stadtteil auf öffentlichen Plätzen sowie auf den FES-Wertstoffhöfen.
Metadaten: {'source': 'data/md_cleaned/was_kommt_

In [4]:
from IPython.display import display, HTML

def print_corrections(original, corrected, emb):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Ähnlichkeits Score: {original}</h3>
        <h3>Inhalt Chunk:</h3>
        <p style="white-space: pre-wrap; color: green;">{corrected}</p>
        <h3>Metadaten: </h3>
        <p style="white-space: pre-wrap; color: yellow;">{emb}</p>
    </div>
    """))
    
# Aufruf der Funktion für jedes Dokument

    
# for document in docs_with_embeddings:
#     print_corrections({document['metadata']['source']}, document['page_content'], document['embeddings'])

for i, (document, metadata, distance) in enumerate(zip(rag_results['documents'][0], rag_results['metadatas'][0], rag_results['distances'][0])):
    similarity_score = 1 / (1 + distance)  # Berechnung des Ähnlichkeitsscores
    # print("Ähnlichkeitsscore:", similarity_score)
    # print(f"Dokument {i+1}:")
    # print("Inhalt:", document)
    # print("Metadaten:", metadata)
    
    print_corrections(similarity_score, document, metadata)


In [16]:
# Initialisiere die Modelle
mistral_model, llama_model = initialize_models()

def create_context_for_llm():

    filter_context = ""
    # Überprüfe den Aufbau des Ergebnisses und gib die Dokumente aus
    if 'documents' in rag_results:
        for i, document in enumerate(rag_results['documents'][0]):
            # print(f"Dokument {i+1}:")
            # print("Inhalt:", document)
            # print()
            filter_context += f"Dokument {i+1}:\n"
            filter_context += f"Inhalt: {document}\n\n"
    else:
        print("Keine ähnlichen Dokumente gefunden.")
        
    return filter_context
    
filter_context = create_context_for_llm()
# print(filter_context)
response_llm = ask_llm_with_ollama(query, filter_context, mistral_model)
print(response_llm)

Basierend auf dem bereitgestellten Kontext ist die Telefonnummer der FES Frankfurter Entsorgungs- und Service GmbH 0800 2008007-0.


In [ ]:
from IPython.display import display, HTML

def print_corrections(query, response):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Frage</h3>
        <p style="white-space: pre-wrap; color: white;">{query}</p>
        <h3>Antwort des LLMs: </h3>
        <p style="white-space: pre-wrap; color: green;">{response}</p>
    </div>
    """))

print_corrections(query, response_llm)


In [19]:
# Initialisiere die Modelle
mistral_model, llama_model = initialize_models()

response_llm_mistal = ask_llm_with_ollama(query, filter_context, mistral_model)
response_llm_llama = ask_llm_with_ollama(query, filter_context, llama_model)

from IPython.display import display, HTML

def print_corrections(query, response_mistral, response_llama):
    display(HTML(f"""
    <div style="border: 1px solid black; padding: 10px; margin: 10px; max-height: 300px; overflow-y: auto;">
        <h3>Frage</h3>
        <p style="white-space: pre-wrap; color: white;">{query}</p>
        <h3>Antwort Mistral: </h3>
        <p style="white-space: pre-wrap; color: green;">{response_mistral}</p>
        <h3>Antwort Llama: </h3>
        <p style="white-space: pre-wrap; color: yellow;">{response_llama}</p>
    </div>
    """))

print_corrections(query, response_llm_mistal, response_llm_llama)